<a href="https://colab.research.google.com/github/smirzap/AI-dengan-DL/blob/main/food101_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRANSFER LEARNING

*Lihat Buku BAB 11*

**Transfer Learning** adalah metode Machine Learning yang memungkinkan model-model yang telah dikembangkan untuk suatu tugas tertentu digunakan kembali sebagai titik awal untuk (membuat) model pada tugas-tugas lain yang serupa.


Tiga strategy Transfer Learning:
1. **Strategi 1** yang melatih ulang semua lapisan atau parameter pada jaringan
2. **Strategi 2** yang melatih sebagian lapisan pada jaringan, yaitu lapisan-lapisan pada bagian belakang dan lapisan FC/DNN.
3. **Strategi 3** yang melatih hanya bagian classifier atau lapisan FC/DNN saja.

Terdapat tiga eksperimen sesuai dengan jenis strategi penggunaan Transfer Learning. Sumber data berasal dari Food101:
* Dipilih 10 kategori makanan (dari 101 kategori) dengan masing-masing kategori berisikan 75 gambar pada training set dan 250 gambar pada test set.
* Sepuluh kategori makanan pada training set adalah chicken_curry, chicken_wings, fried_rice, grilled_salmon, hamburger, ice_cream, pizza, ramen, steak, dan sushi.




## A. Pastikan Pengunaan GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Pastikan Anda menggunakan GPU dengan perintah berikut
!nvidia-smi

Sun Jul 30 10:35:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## B. Unduh Dataset Food101 dengan 10 Label

In [6]:
from zipfile import ZipFile

# Ambil dataset Food101 yang telah ditetapkan 10 kategori makanan
# Kemudian unzip data tersebut

food_file = '/content/drive/MyDrive/10_food_classes_10_percent.zip'
with ZipFile(food_file) as zip:
  zip.extractall()

In [7]:
# Lihat nama-nama direktori yang berisikan dataset
!ls 10_food_classes_10_percent

test  train


In [8]:
dir_path = '10_food_classes_10_percent/'
training_set_dir = dir_path + 'train/'
test_set_dir = dir_path + 'test/'



---


**Catatan Kode di bawah:**

`os.walk()` adalah fungsi untuk mengitari secara rekursif sebuah pohon direktori dan mengembalikan tuple yang terdiri dari 3 komponen, yaitu `dirpath`, `dirnames`, `filenames`, untuk setiap direktori di pohon.
*   `dirpath` adalah jalur ke direktori
*   `dirnames` adalah daftar nama subdirektori di direktori
* `filenames` adalah daftar nama file di direktori

In [9]:
import os

# Mari kita lihat jumlah example pada training set dan test set
for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"Terdapat {len(dirnames)} direktori and {len(filenames)} file gambar di '{dirpath}'.")

Terdapat 2 direktori and 0 file gambar di '10_food_classes_10_percent/'.
Terdapat 10 direktori and 0 file gambar di '10_food_classes_10_percent/test'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/sushi'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/ice_cream'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/grilled_salmon'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/chicken_wings'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/chicken_curry'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/ramen'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/hamburger'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/fried_rice'.
Terdapat 0 direktori and 250 file gambar di '10_food_classes_10_percent/test/steak'.
Terdapat 0 direktori and 250 file gambar di '1

## C. Buat Training dan Test Set

**Catatan kode di bawah:**

`image_dataset_from_directory()` adalah fungsi TensorFlow yang digunakan untuk membuat dataset gambar dari direktori.

In [10]:
# import fungsi untuk penyiapan dataset dalam format TensorFlow
from tensorflow.keras.preprocessing import image_dataset_from_directory

# ukuran gambar input yang akan diproses (tinggi, lebar)
# jika ukuran berbeda maka akan dibuat menjadi ukuran ini
IMAGE_SIZE = (224, 224)

# ukuran batch (jumlah gambar perbatch) yang akan dibuat
# jika gunakan nilai default = 32
# maka batch yang terdiri dari 32 gambar akan dikirimkan ke model pada setiap putaran
BATCH_SIZE = 32

# Penyiapan training set
# Lihat buku untuk penjelasan setiap parameter
training_set = image_dataset_from_directory(directory=training_set_dir,
                                            image_size=IMAGE_SIZE,
                                            label_mode='categorical',
                                            batch_size=BATCH_SIZE,
                                            validation_split=0.3,
                                            subset='training',
                                            seed=42)

# Penyiapan validation set (30% dari training set)
validation_set = image_dataset_from_directory(directory=training_set_dir,
                                            image_size=IMAGE_SIZE,
                                            label_mode='categorical',
                                            batch_size=BATCH_SIZE,
                                            validation_split=0.3,
                                            subset='validation',
                                            seed=42)

# Penyiapan test set
test_set = image_dataset_from_directory(directory=test_set_dir,
                                        image_size=IMAGE_SIZE,
                                        label_mode='categorical')

Found 750 files belonging to 10 classes.
Using 525 files for training.
Found 750 files belonging to 10 classes.
Using 225 files for validation.
Found 2500 files belonging to 10 classes.


In [11]:
# lihat 10 nama kategori makanan pada dataset
training_set.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']



---


**Catatan untuk output di bawah ini:**
* `(None, 224, 224, 3)` mengacu pada bentuk tensor dari gambar kita, di mana `None` adalah ukuran batch, `224` adalah tinggi dan lebar gambar serta `3` adalah jumlah saluran warna (merah, hijau, biru)
* Baik tensor gambar dan label memiliki tipe data `tf.float32`
* Nilai `batch_size `adalah `None` karena hanya digunakan selama pelatihan model. Anda dapat menganggap `None` sebagai placeholder yang menunggu untuk diisi dengan parameter `batch_size` dari fungsi `image_dataset_from_directory()` saat model dilatih
* Penggunaan nilai placeholder `None` untuk `batch_size` memiliki beberapa keuntungan:

  1. Memungkinkan untuk menentukan nilai `batch_size` secara dinamis saat model dilatih
  2. Memungkinkan untuk menggunakan nilai `batch_size` yang berbeda untuk berbagai tahap pelatihan model




In [12]:
# (None, 224, 224, 3) refers to the tensor shape of our images where None is the batch size, 224 is the height (and width) and 3 is the color channels (red, green, blue).
# (None, 10) refers to the tensor shape of the labels where None is the batch size and 10 is the number of possible labels (the 10 different food classes).
# Both image tensors and labels are of the datatype tf.float32.
# The batch_size is None due to it only being used during model training. You can think of None as a placeholder waiting to be filled with the batch_size parameter from image_dataset_from_directory().

training_set

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>



---


**Catatan: eksperimen dimulai dari Strategi 3 diikuti oleh Strategi 2 dan Strategi 1.**

## D. Strategi 3: Hanya Mengubah Output Layer

In [13]:
import tensorflow as tf

tf.random.set_seed(32)

#-------------
# Mari kita buat model dengan metode Transfer Learning
#------------

#1 Pilih dan unduh pretrained model
pretrained_model = tf.keras.applications.EfficientNetV2B0(weights='imagenet',
                                                        include_top=False,
                                                        pooling='avg')

#2 Buat agar paramater model tidak bisa diubah (frozen)
pretrained_model.trainable=False

24274472/24274472 [==============================] - 2s 0us/step


In [14]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

# Buat Input baru karena kita menggunakan training set sendiri yang berbeda dengan ImageNet
inputs = Input(shape=(224,224,3), name='input_layer')

# Berikan Input layer baru sebagai input pada pretrained-model
revised_model_3 = pretrained_model(inputs)

# Definisikan output layer baru (karena kita menggunakan strategi 3)
outputs = Dense(10, activation='softmax', name='output_layer')(revised_model_3)

# Buat Model dengan menghubungkan antara inputs dan outputs
model_3 = Model(inputs, outputs, name='model_3')

In [15]:
# Mari kita compile model_3 agar siap dilakukan
# pelatihan untuk parameter jaringan (bagian output saja pada kasus ini)
model_3.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics='accuracy')

In [16]:
# lihat struktur dan parameter model
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5,932,122
Trainable params: 12,810
Non-trainable params: 5,919,312
_________________________________________________________________


**Catatan kode di bawah ini:**

Perintah `%load_ext tensorboard` adalah perintah Jupyter yang memuat ekstensi TensorBoard ke notebook saat ini. Ekstensi ini memungkinkan Anda untuk memvisualisasi pelatihan dan evaluasi model pembelajaran mesin Anda menggunakan TensorBoard.

In [17]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

# hapus log tensorboard dari hasil eksekusi sebelumnya jika ada
!rm -rf /logs/fit/

# load tensorboard dengan magic command Jupyter
%load_ext tensorboard

# tetapkan direktori penyimpanan data hasil tangkapan
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# buat callback tensorboard
tb_callbacks = TensorBoard(log_dir=log_folder)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# buat EarlyStopping callback
es_callback = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# buat ModelCheckpoint callback
mpt_callback= ModelCheckpoint(filepath='best_model_3.hdf5', save_best_only=True, verbose=1)


In [19]:
# Sekarang kita latih model_3 untuk mendapatkan parameter pada output layer
# Wow, kita mendapat akurasi 83%! dengan menggunakan strategi 3
tf.random.set_seed(42)

import time
start = time.perf_counter()

model_3.fit(training_set,
            epochs=100,
            validation_data=validation_set,
            callbacks = [tb_callbacks, es_callback, mpt_callback])

elapsed = time.perf_counter() - start

Epoch 1/100
17/17 [==============================] - ETA: 0s - loss: 1.9710 - accuracy: 0.3600
Epoch 1: val_loss improved from inf to 1.57035, saving model to best_model_3.hdf5
17/17 [==============================] - 21s 307ms/step - loss: 1.9710 - accuracy: 0.3600 - val_loss: 1.5704 - val_accuracy: 0.6533
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 1.3284 - accuracy: 0.6953
Epoch 2: val_loss improved from 1.57035 to 1.18953, saving model to best_model_3.hdf5
17/17 [==============================] - 4s 185ms/step - loss: 1.3211 - accuracy: 0.6971 - val_loss: 1.1895 - val_accuracy: 0.7111
Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.9834 - accuracy: 0.7871
Epoch 3: val_loss improved from 1.18953 to 0.99460, saving model to best_model_3.hdf5
17/17 [==============================] - 3s 128ms/step - loss: 0.9851 - accuracy: 0.7829 - val_loss: 0.9946 - val_accuracy: 0.7333
Epoch 4/100
16/17 [===========================>..] - ETA: 0s - loss

In [20]:
print('Elapsed %.3f seconds.' % elapsed)

Elapsed 115.620 seconds.


In [ ]:
# Harap sabar: tensorboard memerlukan waktu untuk tayang
%tensorboard --logdir={log_folder}

In [22]:
from tensorflow.keras.models import load_model

# Load best hyperparameters dari hasil pembelajaran model yang telah dilakukan
best_model_3 = load_model('best_model_3.hdf5')

# Lakukan evaluasi kinerja model dengan test set
best_model_3.evaluate(test_set)

79/79 [==============================] - 8s 74ms/step - loss: 0.4098 - accuracy: 0.8704


[0.4097979962825775, 0.8704000115394592]

## E. Strategi 2: Trainable 10 Layer

In [24]:
# Set pretrained_model agar parameter jaringan bisa dilatih
pretrained_model.trainable=True

In [25]:
print(len(pretrained_model.trainable_variables))

241


In [26]:
# Perhatikan bahwa semua layer pada pretrained model telah di frozen!
for urutan, lapisan in enumerate(pretrained_model.layers):
  print(urutan, lapisan.name, lapisan.trainable)

0 input_1 True
1 rescaling True
2 normalization True
3 stem_conv True
4 stem_bn True
5 stem_activation True
6 block1a_project_conv True
7 block1a_project_bn True
8 block1a_project_activation True
9 block2a_expand_conv True
10 block2a_expand_bn True
11 block2a_expand_activation True
12 block2a_project_conv True
13 block2a_project_bn True
14 block2b_expand_conv True
15 block2b_expand_bn True
16 block2b_expand_activation True
17 block2b_project_conv True
18 block2b_project_bn True
19 block2b_drop True
20 block2b_add True
21 block3a_expand_conv True
22 block3a_expand_bn True
23 block3a_expand_activation True
24 block3a_project_conv True
25 block3a_project_bn True
26 block3b_expand_conv True
27 block3b_expand_bn True
28 block3b_expand_activation True
29 block3b_project_conv True
30 block3b_project_bn True
31 block3b_drop True
32 block3b_add True
33 block4a_expand_conv True
34 block4a_expand_bn True
35 block4a_expand_activation True
36 block4a_dwconv2 True
37 block4a_bn True
38 block4a_activ

In [27]:
# sekarang kita unfrozen 10 layer pada bagian belakang lapisan convolution
for urutan, lapisan in enumerate(pretrained_model.layers[:-5]):
  lapisan.trainable=False

In [28]:
print(len(pretrained_model.trainable_variables))

3


In [29]:
# Berikan input layer sebagai input pada pretrained-model
revised_model_2 = pretrained_model(inputs)

# Definisikan output layer (strategi 2)
outputs = Dense(10, activation='softmax', name='output_layer')(revised_model_2)

# defisinikan model dengan strategi 3 dengan me-link inputs dan outputs
model_2 = Model(inputs, outputs, name='model_2')

In [30]:
# Mari kita compile model_2 agar siap dilakukan pelatihan untuk parameter jaringan
model_2.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(), #learning_rate=0.0001
                metrics='accuracy')

In [31]:
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5,932,122
Trainable params: 261,130
Non-trainable params: 5,670,992
_________________________________________________________________


In [32]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tb_callbacks = TensorBoard(log_dir=log_folder)

In [33]:
mpt_callback= ModelCheckpoint(filepath='best_model_2.hdf5', save_best_only=True, verbose=1)

In [34]:
# Sekarang kita latih model_3 untuk mendapatkan parameter pada output layer
# Wow, kita mendapat akurasi 83%! dengan menggunakan strategi 3

tf.random.set_seed(42)

start = time.perf_counter()

model_2.fit(training_set,
            epochs=100,
            validation_data=validation_set,
            callbacks = [tb_callbacks, es_callback, mpt_callback])

elapsed = time.perf_counter() - start

Epoch 1/100
16/17 [===========================>..] - ETA: 0s - loss: 1.6163 - accuracy: 0.5371
Epoch 1: val_loss improved from inf to 0.89253, saving model to best_model_2.hdf5
17/17 [==============================] - 14s 255ms/step - loss: 1.5997 - accuracy: 0.5467 - val_loss: 0.8925 - val_accuracy: 0.7689
Epoch 2/100
16/17 [===========================>..] - ETA: 0s - loss: 0.6698 - accuracy: 0.8301
Epoch 2: val_loss improved from 0.89253 to 0.65807, saving model to best_model_2.hdf5
17/17 [==============================] - 3s 128ms/step - loss: 0.6651 - accuracy: 0.8343 - val_loss: 0.6581 - val_accuracy: 0.8222
Epoch 3/100
16/17 [===========================>..] - ETA: 0s - loss: 0.4291 - accuracy: 0.9082
Epoch 3: val_loss improved from 0.65807 to 0.58617, saving model to best_model_2.hdf5
17/17 [==============================] - 3s 153ms/step - loss: 0.4333 - accuracy: 0.9067 - val_loss: 0.5862 - val_accuracy: 0.8489
Epoch 4/100
16/17 [===========================>..] - ETA: 0s - loss

In [35]:
print('Elapsed %.3f seconds.' % elapsed)

Elapsed 63.642 seconds.


In [ ]:
# Gunakan port berbeda (8008) untuk tayangkan tensorboard Strategi 2
%tensorboard --logdir={log_folder} --port=8008

In [38]:
# Load best hyperparameters from previous fit process
best_model_2 = load_model('best_model_2.hdf5')

# Let's evaluate the model
best_model_2.evaluate(test_set)

79/79 [==============================] - 9s 85ms/step - loss: 0.3338 - accuracy: 0.8880


[0.333753377199173, 0.8880000114440918]

In [39]:
class_names = test_set.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

## F. Strategi 1: Seluruh Paramater Dilatih

In [40]:
# Set pretrained_model agar parameter jaringan bisa dilatih
pretrained_model.trainable=True

In [41]:
# Perhatikan bahwa semua layer pada pretrained model telah di frozen!
for urutan, lapisan in enumerate(pretrained_model.layers):
  print(urutan, lapisan.name, lapisan.trainable)

0 input_1 True
1 rescaling True
2 normalization True
3 stem_conv True
4 stem_bn True
5 stem_activation True
6 block1a_project_conv True
7 block1a_project_bn True
8 block1a_project_activation True
9 block2a_expand_conv True
10 block2a_expand_bn True
11 block2a_expand_activation True
12 block2a_project_conv True
13 block2a_project_bn True
14 block2b_expand_conv True
15 block2b_expand_bn True
16 block2b_expand_activation True
17 block2b_project_conv True
18 block2b_project_bn True
19 block2b_drop True
20 block2b_add True
21 block3a_expand_conv True
22 block3a_expand_bn True
23 block3a_expand_activation True
24 block3a_project_conv True
25 block3a_project_bn True
26 block3b_expand_conv True
27 block3b_expand_bn True
28 block3b_expand_activation True
29 block3b_project_conv True
30 block3b_project_bn True
31 block3b_drop True
32 block3b_add True
33 block4a_expand_conv True
34 block4a_expand_bn True
35 block4a_expand_activation True
36 block4a_dwconv2 True
37 block4a_bn True
38 block4a_activ

In [42]:
# Berikan input layer sebagai input pada pretrained-model
revised_model_1 = pretrained_model(inputs)

# Definisikan output layer (strategi 1)
outputs = Dense(10, activation='softmax', name='output_layer')(revised_model_1)

# defisinikan model dengan strategi 3 dengan me-link inputs dan outputs
model_1 = Model(inputs, outputs, name='model_1')

In [43]:
# Mari kita compile model_1 agar siap dilakukan pelatihan untuk parameter jaringan
model_1.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(), #learning_rate=0.0001
                metrics='accuracy')

In [44]:
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tb_callbacks = TensorBoard(log_dir=log_folder)

In [45]:
mpt_callback= ModelCheckpoint(filepath='best_model_1.hdf5', save_best_only=True, verbose=1)

In [46]:
# Sekarang kita latih model_3 untuk mendapatkan parameter pada output layer
# Wow, kita mendapat akurasi 83%! dengan menggunakan strategi 3

tf.random.set_seed(42)

start = time.perf_counter()

model_1.fit(training_set,
            epochs=100,
            validation_data=validation_set,
            callbacks = [tb_callbacks, mpt_callback, es_callback])

elapsed = time.perf_counter() - start

Epoch 1/100
17/17 [==============================] - ETA: 0s - loss: 1.5730 - accuracy: 0.5048
Epoch 1: val_loss improved from inf to 0.89514, saving model to best_model_1.hdf5
17/17 [==============================] - 62s 466ms/step - loss: 1.5730 - accuracy: 0.5048 - val_loss: 0.8951 - val_accuracy: 0.7067
Epoch 2/100
17/17 [==============================] - ETA: 0s - loss: 0.3729 - accuracy: 0.9029
Epoch 2: val_loss improved from 0.89514 to 0.83861, saving model to best_model_1.hdf5
17/17 [==============================] - 5s 274ms/step - loss: 0.3729 - accuracy: 0.9029 - val_loss: 0.8386 - val_accuracy: 0.6933
Epoch 3/100
17/17 [==============================] - ETA: 0s - loss: 0.1276 - accuracy: 0.9695
Epoch 3: val_loss did not improve from 0.83861
17/17 [==============================] - 5s 262ms/step - loss: 0.1276 - accuracy: 0.9695 - val_loss: 0.9204 - val_accuracy: 0.7244
Epoch 4/100
17/17 [==============================] - ETA: 0s - loss: 0.0923 - accuracy: 0.9733
Epoch 4: va

In [47]:
print(len(pretrained_model.trainable_variables))

241


In [48]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1280)             5919312   
 al)                                                             
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 5,932,122
Trainable params: 5,871,514
Non-trainable params: 60,608
_________________________________________________________________


In [ ]:
%tensorboard --logdir={log_folder} --port=8010

In [50]:
# Load best hyperparameters from previous fit process
best_model_1 = load_model('best_model_1.hdf5')

# Let's evaluate the model
best_model_1.evaluate(test_set)

79/79 [==============================] - 9s 74ms/step - loss: 0.6562 - accuracy: 0.7896


[0.6561955809593201, 0.7896000146865845]

## F. Klasifikasi Foto Makanan Aktual


In [53]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.io.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

In [54]:
class_names = test_set.class_names
class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [55]:
import os
from zipfile import ZipFile

# Ambil dataset dan unzip data tersebut
!wget https://raw.githubusercontent.com/smirzap/AI-dengan-DL/main/data/my_food_collection.zip

with ZipFile('my_food_collection.zip') as zip:
  zip.extractall()

--2023-07-30 11:06:09--  https://raw.githubusercontent.com/smirzap/AI-dengan-DL/main/data/my_food_collection.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2207554 (2.1M) [application/zip]
Saving to: ‘my_food_collection.zip’

my_food_collection. 100%[===================>]   2.10M  --.-KB/s    in 0.02s   

2023-07-30 11:06:10 (137 MB/s) - ‘my_food_collection.zip’ saved [2207554/2207554]



In [56]:
# Buat list nama file berikut nama direktorinya
custom_food_images = ["my_food_collection/" + foto for foto in os.listdir("my_food_collection")]
custom_food_images

['my_food_collection/sushi 3.jpeg',
 'my_food_collection/sushi 1.JPG',
 'my_food_collection/ramen 2.jpeg',
 'my_food_collection/ramen 1.jpeg',
 'my_food_collection/beef burger.jpg',
 'my_food_collection/sushi 2.JPG']

In [57]:
# Load best hyperparameters from previous fit process
best_model_2 = load_model('best_model_2.hdf5')

In [ ]:
import matplotlib.pyplot as plt

# Make predictions on custom food images
for img in custom_food_images:

  # Read in the image
  img = tf.io.read_file(img)

  # Decode it into a tensor
  img = tf.io.decode_image(img)

  # Resize the image
  img = tf.image.resize(img, [224, 224])

  pred_prob = best_model_2.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [None, 224, 224, 3]
  pred_catg = pred_prob.argmax(axis=-1)
  pred_class = class_names[pred_catg[0]] # find the predicted class label

  # Plot the image with appropriate annotations
  plt.figure()
  plt.imshow(img/255.) # imshow() requires float inputs to be normalized
  plt.title(f"prediksi: {pred_class} ({pred_prob.max()*100:.2f}%)")
  plt.axis(False)

In [ ]:
# Make predictions on custom food images
for img in custom_food_images:
  img = load_and_prep_image(img, scale=False) # load in target image and turn it into tensor
  pred_prob = best_model_3.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [None, 224, 224, 3]
  pred_class = class_names[pred_prob.argmax()] # find the predicted class label
  # Plot the image with appropriate annotations
  plt.figure(dpi=100)
  plt.imshow(img/255.) # imshow() requires float inputs to be normalized
  plt.title(f"prediksi: {pred_class} ({pred_prob.max()*100:.2f}%)")
  plt.axis(False)